In [1]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import ExpectationMaximization as EM
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination, BeliefPropagation
from pgmpy.inference.CausalInference import CausalInference
import pickle

In [41]:
output_dir = 'D:/Thesis/files_output_dir/output_preprocessing/'
ground_truth_level1 = pd.read_csv(output_dir + 'ground_truth_level1.csv')
ground_truth_level2 = pd.read_csv(output_dir + 'ground_truth_level2.csv')
ground_truth_level3 = pd.read_csv(output_dir + 'ground_truth_level3.csv')

In [15]:
ground_truth_level1.head()

,road id,level,lane,dir,Day_type,time_slot,speed,volume/min/lane,volume_cls
0,8651,1,4,0,workday,103,0.898789,30.200000,4
1,8651,1,4,0,workday,63,3.973160,32.450000,5
2,8651,1,4,0,workday,112,1.423980,15.700000,1
3,16710,1,3,0,workday,56,4.619120,12.666667,1
4,8651,1,4,0,workday,80,9.817830,32.100000,5


In [42]:
ground_truth_level1['lane'].value_counts(dropna=False)

lane
4    58
3    18
2    10
Name: count, dtype: int64

## Discritization

In [43]:
def discreteLane(lane):
    if lane < 2:
        return 0
    if lane < 3:
        return 1
    else:
        return 2
    

def discreteTime(timeSlot):
    if timeSlot < 7 * 6: #hour * hourly time slot num.
        return 0
    if timeSlot < 10 *6:
        return 1
    if timeSlot < 16 * 6:
        return 2
    if timeSlot < 20 * 6:
        return 3
    return 4

# discretize the average speed
def discreteSpeed(speed, level):
    if level == 0 or level == 1:
        if speed < 10:
            return 0
        if speed < 15:
            return 1
        if speed < 17:
            return 2
        if speed < 19:
            return 3
        if speed < 21:
            return 4
        return 5
    else:
        if level == 2:
            if speed < 4:
                return 0
            if speed < 6:
                return 1
            if speed < 8:
                return 2
            if speed < 10:
                return 3
            if speed < 12:
                return 4
            return 5
        else:
            if speed < 2:
                return 0
            if speed < 3.5:
                return 1
            if speed < 5:
                return 2
            if speed < 6.5:
                return 3
            if speed < 8:
                return 4
            return 5

In [5]:
ground_truth_level1.columns

Index(['road id', 'level', 'lane', 'dir', 'Day_type', 'time_slot', 'speed',
       'volume/min/lane', 'volume_cls'],
      dtype='object')

In [44]:
level=1

column_function_mapping = {
    'lane': discreteLane,
    'time_slot': discreteTime,
    'speed': lambda x: discreteSpeed(x, level)
}

# Iterate over the mapping and apply the functions to each column
for column, func in column_function_mapping.items():
    ground_truth_level1[column] = ground_truth_level1[column].apply(func)
    ground_truth_level1[column] += 1


ground_truth_level1['volume_cls'] = ground_truth_level1['volume_cls'] -1 
ground_truth_level1['dir'] = ground_truth_level1['dir']  + 1

In [45]:
level=2

column_function_mapping = {
    'lane': discreteLane,
    'time_slot': discreteTime,
    'speed': lambda x: discreteSpeed(x, level)
}

# Iterate over the mapping and apply the functions to each column
for column, func in column_function_mapping.items():
    ground_truth_level2[column] = ground_truth_level2[column].apply(func)
    ground_truth_level2[column] += 1


ground_truth_level2['volume_cls'] = ground_truth_level2['volume_cls'] -1 
ground_truth_level2['dir'] = ground_truth_level2['dir']  + 1

In [46]:
level=3

column_function_mapping = {
    'lane': discreteLane,
    'time_slot': discreteTime,
    'speed': lambda x: discreteSpeed(x, level)
}

# Iterate over the mapping and apply the functions to each column
for column, func in column_function_mapping.items():
    ground_truth_level3[column] = ground_truth_level3[column].apply(func)
    ground_truth_level3[column] += 1


ground_truth_level3['volume_cls'] = ground_truth_level3['volume_cls'] -1 
ground_truth_level3['dir'] = ground_truth_level3['dir']  + 1

In [47]:
# Getting unique values and their counts
unique_values_counts = {col: (ground_truth_level1[col].unique(), ground_truth_level1[col].nunique()) 
                        for col in ground_truth_level1.columns}

# Printing the unique values and counts in a nicely formatted way
for column, (unique_vals, count) in unique_values_counts.items():
    print(f"Column: {column}")
    print(f"Unique Values: {unique_vals}")
    print(f"Number of Unique Values: {count}")
    print("-" * 50)  # Separator for readability

Column: road id
Unique Values: [ 8651 16710 16615 16702 16659 16657 23613 23611 16662 16708 16617 16704]
Number of Unique Values: 12
--------------------------------------------------
Column: level
Unique Values: [1]
Number of Unique Values: 1
--------------------------------------------------
Column: lane
Unique Values: [3 2]
Number of Unique Values: 2
--------------------------------------------------
Column: dir
Unique Values: [1]
Number of Unique Values: 1
--------------------------------------------------
Column: Day_type
Unique Values: ['workday' 'holiday']
Number of Unique Values: 2
--------------------------------------------------
Column: time_slot
Unique Values: [4 3 2 5]
Number of Unique Values: 4
--------------------------------------------------
Column: speed
Unique Values: [1 2 6 3]
Number of Unique Values: 4
--------------------------------------------------
Column: volume/min/lane
Unique Values: [30.2        32.45       15.7        12.66666667 32.1        29.8
 26.86666

In [48]:
# Getting unique values and their counts
unique_values_counts = {col: (ground_truth_level2[col].unique(), ground_truth_level2[col].nunique()) 
                        for col in ground_truth_level2.columns}

# Printing the unique values and counts in a nicely formatted way
for column, (unique_vals, count) in unique_values_counts.items():
    print(f"Column: {column}")
    print(f"Unique Values: {unique_vals}")
    print(f"Number of Unique Values: {count}")
    print("-" * 50)  # Separator for readability

Column: road id
Unique Values: [14540 17058  5252 17904 15664 15585 15862  9142 12824  8745 15768 15595
  5237  9485 12820 12909 12742 15773 12371 17582 14541 12372  9040 15544
 17846 15657 12825  9362 12357 12821 12908 12743 14681]
Number of Unique Values: 33
--------------------------------------------------
Column: level
Unique Values: [2]
Number of Unique Values: 1
--------------------------------------------------
Column: lane
Unique Values: [2 3]
Number of Unique Values: 2
--------------------------------------------------
Column: dir
Unique Values: [1]
Number of Unique Values: 1
--------------------------------------------------
Column: Day_type
Unique Values: ['workday' 'holiday']
Number of Unique Values: 2
--------------------------------------------------
Column: time_slot
Unique Values: [3 4 2 5]
Number of Unique Values: 4
--------------------------------------------------
Column: speed
Unique Values: [1 5 3 2 4]
Number of Unique Values: 5
-----------------------------------

In [49]:
# Getting unique values and their counts
unique_values_counts = {col: (ground_truth_level3[col].unique(), ground_truth_level3[col].nunique()) 
                        for col in ground_truth_level3.columns}

# Printing the unique values and counts in a nicely formatted way
for column, (unique_vals, count) in unique_values_counts.items():
    print(f"Column: {column}")
    print(f"Unique Values: {unique_vals}")
    print(f"Number of Unique Values: {count}")
    print("-" * 50)  # Separator for readability

Column: road id
Unique Values: [17581 17977  4587 12809]
Number of Unique Values: 4
--------------------------------------------------
Column: level
Unique Values: [3]
Number of Unique Values: 1
--------------------------------------------------
Column: lane
Unique Values: [2 3 1]
Number of Unique Values: 3
--------------------------------------------------
Column: dir
Unique Values: [2]
Number of Unique Values: 1
--------------------------------------------------
Column: Day_type
Unique Values: ['workday' 'holiday']
Number of Unique Values: 2
--------------------------------------------------
Column: time_slot
Unique Values: [4 3 2 5]
Number of Unique Values: 4
--------------------------------------------------
Column: speed
Unique Values: [1 3 2 5 4 6]
Number of Unique Values: 6
--------------------------------------------------
Column: volume/min/lane
Unique Values: [3.4        5.8        7.4        8.6        5.7        8.
 6.9        6.5        6.6        7.2        0.86666667 0.3

In [6]:
# # Define the Bayesian Network structure with latent variables
# model = BayesianNetwork([
#     ('length', 'theta'),
#     ('lanes', 'theta'),
#     ('max_speed', 'theta'),
#     ('dir', 'theta'),
#     ('tor', 'theta'),
#     ('n_connnections', 'theta'),
#     ('weighted_POI', 'theta'),
#     ('theta', 'Volume'),
#     ('theta', 'avg_speed'),
#     ('time', 'Volume'),
#     ('time', 'avg_speed'),
#     ('no_taxi_car', 'Volume'),
#     ('no_taxi_car', 'avg_speed std'),
#     ('avg_speed', 'avg_speed std')
# ])
# model.latents = {'theta', 'Volume'}

In [8]:
# # Initialize the Expectation-Maximization (EM) estimator
# data = level3_road_history_workday.copy()  # Make sure to call the copy() method correctly
# estimator = EM(model, data)

In [9]:
# estimated_params = estimator.get_parameters(latent_card={'theta': 3, 'Volume': 5})  # Specify cardinalities of latent variables

# # After estimating parameters, add the learned CPDs back to the model
# for cpd in estimated_params:
#     model.add_cpds(cpd)
# # Validate the model (check if the model is correctly specified)
# assert model.check_model()
# # Save the model to a file
# output_dir = "D:/Thesis/files_output_dir/output_files/"
# with open(output_dir + 'bayesian_network_model_level3_workday.pkl', 'wb') as file:
#     pickle.dump(model, file)

100%|██████████| 100/100 [4:57:38<00:00, 178.59s/it] 


## Load Model

In [26]:
file_path = "D:/Thesis/files_output_dir/output_files/"

## Level-1

In [54]:
# Load the model
with open(file_path + 'bayesian_network_model_level1_workday.pkl', 'rb') as file:
    model_workday = pickle.load(file)
    
# Load the model
with open(file_path + 'bayesian_network_model_level1_holiday.pkl', 'rb') as file:
    model_holiday = pickle.load(file)

Learned CPD for tor:
+--------+-----------+
| tor(1) | 0.974973  |
+--------+-----------+
| tor(2) | 0.0250273 |
+--------+-----------+


Learned CPD for avg_speed std:
+------------------+-----+----------------------+
| avg_speed        | ... | avg_speed(6)         |
+------------------+-----+----------------------+
| no_taxi_car      | ... | no_taxi_car(7)       |
+------------------+-----+----------------------+
| avg_speed std(1) | ... | 0.01099332548095799  |
+------------------+-----+----------------------+
| avg_speed std(2) | ... | 0.4166121362823365   |
+------------------+-----+----------------------+
| avg_speed std(3) | ... | 0.4189242245779348   |
+------------------+-----+----------------------+
| avg_speed std(4) | ... | 0.11730576277101601  |
+------------------+-----+----------------------+
| avg_speed std(5) | ... | 0.027483313702394974 |
+------------------+-----+----------------------+
| avg_speed std(6) | ... | 0.008681237185359682 |
+------------------+-----+-----

In [55]:
# Retrieve and print the learned CPDs
for cpd in model_workday.get_cpds():
    print(f"Learned CPD for {cpd.variable}:")
    print(cpd)
    print("\n")
    
# Retrieve and print the learned CPDs
for cpd in model_holiday.get_cpds():
    print(f"Learned CPD for {cpd.variable}:")
    print(cpd)
    print("\n")

Learned CPD for time:
+---------+----------+
| time(1) | 0.255789 |
+---------+----------+
| time(2) | 0.133219 |
+---------+----------+
| time(3) | 0.265752 |
+---------+----------+
| time(4) | 0.178212 |
+---------+----------+
| time(5) | 0.167029 |
+---------+----------+


Learned CPD for no_taxi_car:
+----------------+-----------+
| no_taxi_car(2) | 0.0727021 |
+----------------+-----------+
| no_taxi_car(3) | 0.0805577 |
+----------------+-----------+
| no_taxi_car(4) | 0.0771346 |
+----------------+-----------+
| no_taxi_car(5) | 0.112501  |
+----------------+-----------+
| no_taxi_car(6) | 0.174575  |
+----------------+-----------+
| no_taxi_car(7) | 0.48253   |
+----------------+-----------+


Learned CPD for n_connnections:
+-------------------+-----------+
| n_connnections(1) | 0.574967  |
+-------------------+-----------+
| n_connnections(2) | 0.418013  |
+-------------------+-----------+
| n_connnections(3) | 0.0070203 |
+-------------------+-----------+


Learned CPD for t

In [29]:
ground_truth_level1.columns

Index(['road id', 'level', 'lane', 'dir', 'Day_type', 'time_slot', 'speed',
       'volume/min/lane', 'volume_cls'],
      dtype='object')

In [57]:
# Perform inference using VariableElimination
inference_workday = BeliefPropagation(model_workday)
inference_holiday = BeliefPropagation(model_holiday)
# inference = VariableElimination(model)
ground_truth_level1['theta'] = np.nan

# Iterate through each row in the data to perform inference
for index, row in ground_truth_level1.iterrows():
    # Extract observed evidence from the current row
    evidence = {
        'lanes': row['lane'],
        'dir': row['dir'],
        'time': row['time_slot'],
        'Volume' : row['volume_cls'],
        'avg_speed': row['speed']
        
    }
    
    # Remove NaN values from the evidence dictionary
    evidence = {k: v for k, v in evidence.items() if pd.notna(v)}

    print(index, evidence)
    
    # Perform MAP query to find the most likely value of latent nodes
    if row['Day_type'] == 'workday':
        result_theta = inference_workday.map_query(variables=['theta'], evidence=evidence)
    else:
        result_theta = inference_holiday.map_query(variables=['theta'], evidence=evidence)
    
    
    # Insert the inferred values into the corresponding columns
    ground_truth_level1.at[index, 'theta'] = result_theta.get('theta', np.nan)
    print(ground_truth_level1.at[index, 'theta'])
 

0 {'lanes': 3, 'dir': 1, 'time': 4, 'Volume': 3, 'avg_speed': 1}
2.0
1 {'lanes': 3, 'dir': 1, 'time': 3, 'Volume': 4, 'avg_speed': 1}
2.0
2 {'lanes': 3, 'dir': 1, 'time': 4, 'Volume': 0, 'avg_speed': 1}
2.0
3 {'lanes': 3, 'dir': 1, 'time': 2, 'Volume': 0, 'avg_speed': 1}
2.0
4 {'lanes': 3, 'dir': 1, 'time': 3, 'Volume': 4, 'avg_speed': 1}
2.0
5 {'lanes': 3, 'dir': 1, 'time': 4, 'Volume': 3, 'avg_speed': 1}
0.0
6 {'lanes': 3, 'dir': 1, 'time': 2, 'Volume': 2, 'avg_speed': 1}
2.0
7 {'lanes': 3, 'dir': 1, 'time': 3, 'Volume': 3, 'avg_speed': 1}
2.0
8 {'lanes': 3, 'dir': 1, 'time': 4, 'Volume': 1, 'avg_speed': 1}
2.0
9 {'lanes': 3, 'dir': 1, 'time': 4, 'Volume': 3, 'avg_speed': 1}
0.0
10 {'lanes': 3, 'dir': 1, 'time': 5, 'Volume': 0, 'avg_speed': 1}
2.0
11 {'lanes': 3, 'dir': 1, 'time': 5, 'Volume': 0, 'avg_speed': 1}
2.0
12 {'lanes': 2, 'dir': 1, 'time': 3, 'Volume': 1, 'avg_speed': 1}
1.0
13 {'lanes': 3, 'dir': 1, 'time': 3, 'Volume': 0, 'avg_speed': 1}
2.0
14 {'lanes': 2, 'dir': 1, 'tim

In [58]:
# Print the first few rows of the updated DataFrame
ground_truth_level1.head()

,road id,level,lane,dir,Day_type,time_slot,speed,volume/min/lane,volume_cls,theta
0,8651,1,3,1,workday,4,1,30.200000,3,2.0
1,8651,1,3,1,workday,3,1,32.450000,4,2.0
2,8651,1,3,1,workday,4,1,15.700000,0,2.0
3,16710,1,3,1,workday,2,1,12.666667,0,2.0
4,8651,1,3,1,workday,3,1,32.100000,4,2.0


In [59]:
ground_truth_level1['theta'].value_counts(dropna=False)

theta
2.0    65
0.0    11
1.0    10
Name: count, dtype: int64

## Level-2

In [ ]:
# Load the model
with open(file_path + 'bayesian_network_model_level2_workday.pkl', 'rb') as file:
    model_workday = pickle.load(file)
    
# Load the model
with open(file_path + 'bayesian_network_model_level2_holiday.pkl', 'rb') as file:
    model_holiday = pickle.load(file)

# Retrieve and print the learned CPDs
for cpd in model_workday.get_cpds():
    print(f"Learned CPD for {cpd.variable}:")
    print(cpd)
    print("\n")
    
# Retrieve and print the learned CPDs
for cpd in model_holiday.get_cpds():
    print(f"Learned CPD for {cpd.variable}:")
    print(cpd)
    print("\n")

# Perform inference using VariableElimination
inference_workday = BeliefPropagation(model_workday)
inference_holiday = BeliefPropagation(model_holiday)

# inference = VariableElimination(model)
ground_truth_level2['theta'] = np.nan

# Iterate through each row in the data to perform inference
for index, row in ground_truth_level2.iterrows():
    # Extract observed evidence from the current row
    evidence = {
        'lanes': row['lane'],
        'dir': row['dir'],
        'time': row['time_slot'],
        'Volume' : row['volume_cls'],
        'avg_speed': row['speed']
        
    }
    
    # Remove NaN values from the evidence dictionary
    evidence = {k: v for k, v in evidence.items() if pd.notna(v)}

    print(index, evidence)
    
    # Perform MAP query to find the most likely value of latent nodes
    if row['Day_type'] == 'workday':
        result_theta = inference_workday.map_query(variables=['theta'], evidence=evidence)
    else:
        result_theta = inference_holiday.map_query(variables=['theta'], evidence=evidence)
    
    
    # Insert the inferred values into the corresponding columns
    ground_truth_level2.at[index, 'theta'] = result_theta.get('theta', np.nan)
    print(ground_truth_level2.at[index, 'theta'])
 

## Save Data

In [60]:
output_dir = "D:/Thesis/files_output_dir/output_files/"
ground_truth_level1.to_csv(output_dir + 'ground_truth_level1_discritize_data.csv', index=False)